<a href="https://colab.research.google.com/github/Mohbajry-77/RSA_Homorphic_Lap1-Sarah-Bajry/blob/main/SMPC_Lab4_Assignment_Sara%20Bajry_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# File Name: MPC_Lab_Sarah_Saleh_Part1.py
# Student Name: Sarah Saleh
# Description: Part I - MPC Sum using Secret Sharing

import random

class Participant:
    """
    Represents a participant in the MPC protocol.
    Each participant has a name and a private secret value (e.g., salary).
    """
    def __init__(self, name, secret_value):
        self.name = name                # Participant's name
        self.secret_value = secret_value  # The private data to be protected
        self.shares = []                # Shares generated from the secret
        self.received_shares = []       # Shares received from other participants

    def split_secret(self, num_participants):
        """
        Splits the secret value into a number of random shares.
        The sum of these shares must equal the original secret value.
        """
        # Generate (n-1) random shares. n is the number of participants.
        temp_shares = [random.randint(1, 1000) for _ in range(num_participants - 1)]

        # Calculate the last share to ensure the total sum matches the secret.
        final_share = self.secret_value - sum(temp_shares)

        # Combine all shares into one list and shuffle it.
        self.shares = temp_shares + [final_share]
        random.shuffle(self.shares)

    def sum_received_shares(self):
        """Sums all the shares this participant has received from others."""
        return sum(self.received_shares)

# --- Simulation of the MPC Sum Protocol ---
print("--- Part 1: MPC Sum Simulation ---")

# 1. Initialize participants with their secret values.
participants_data = {"Ahmed": 80, "Ali": 50, "Rashida": 90}
participants = [Participant(name, value) for name, value in participants_data.items()]
num_participants = len(participants)
print(f"Original secrets (for verification only): {participants_data}")

# 2. Each participant splits their secret value into shares.
for p in participants:
    p.split_secret(num_participants)

# 3. Distribute the shares among all participants.
for i, sender in enumerate(participants):
    for j, receiver in enumerate(participants):
        share_to_send = sender.shares[j]
        receiver.received_shares.append(share_to_send)

# 4. Each participant sums the shares they have received.
partial_sums = []
for p in participants:
    partial_sum = p.sum_received_shares()
    partial_sums.append(partial_sum)
    print(f"{p.name}'s partial sum is: {partial_sum}")

# 5. Compute the final global sum by adding the partial sums.
global_sum = sum(partial_sums)
print(f"\nFinal Computed Sum: {global_sum}")

# Verification step
original_sum = sum(p.secret_value for p in participants)
assert global_sum == original_sum
print(f"Verification Successful: The computed sum ({global_sum}) matches the original sum ({original_sum}).")

--- Part 1: MPC Sum Simulation ---
Original secrets (for verification only): {'Ahmed': 80, 'Ali': 50, 'Rashida': 90}
Ahmed's partial sum is: 1908
Ali's partial sum is: -153
Rashida's partial sum is: -1535

Final Computed Sum: 220
Verification Successful: The computed sum (220) matches the original sum (220).


In [ ]:

# File Name: MPC_Lab_Sarah_Saleh_Part2.py
# Student Name: Sarah Saleh
# Description: Part II - Shamir's Secret Sharing Implementation

import random
# We need the 'sympy' library for modular inverse. Colab has it pre-installed.
from sympy import mod_inverse

# A large prime number for the finite field arithmetic.
PRIME = 2**127 - 1

def generate_shares(secret, n, k):
    """
    Generates 'n' shares from a secret, with a threshold 'k'.
    'k' is the minimum number of shares needed to reconstruct the secret.
    """
    # Create a random polynomial of degree k-1. The secret is the constant term.
    coefficients = [secret] + [random.randint(1, PRIME - 1) for _ in range(k - 1)]

    # Generate 'n' points (shares) on this polynomial.
    shares = []
    for i in range(1, n + 1):
        x = i
        # Evaluate polynomial at point x: y = f(x)
        y = 0
        for j in range(len(coefficients)):
            y = (y + coefficients[j] * (x ** j)) % PRIME
        shares.append((x, y))
    return shares

def reconstruct_secret(shares, k):
    """
    Reconstructs the secret from a list of 'k' shares using Lagrange Interpolation.
    The secret is the value of the polynomial at x=0.
    """
    if len(shares) < k:
        raise ValueError(f"Need at least {k} shares to reconstruct the secret.")

    # Use only the first k shares from the provided list.
    points = shares[:k]

    secret = 0
    # Combine the points to find the y-intercept (the secret)
    for i, (xi, yi) in enumerate(points):
        # Calculate the Lagrange basis polynomial term for this point
        numerator, denominator = 1, 1
        for j, (xj, _) in enumerate(points):
            if i != j:
                numerator = (numerator * -xj) % PRIME
                denominator = (denominator * (xi - xj)) % PRIME

        # We need the modular multiplicative inverse of the denominator.
        inv_denominator = mod_inverse(denominator, PRIME)
        term = (yi * numerator * inv_denominator) % PRIME
        secret = (secret + term) % PRIME

    return secret

# --- Practical Example ---
SECRET_VALUE = 1234
NUM_SHARES = 6  # n: total number of shares
THRESHOLD = 3   # k: minimum shares required for reconstruction

print("\n--- Part 2: Shamir's Secret Sharing ---")
print(f"Original Secret: {SECRET_VALUE}, Total Shares (n): {NUM_SHARES}, Threshold (k): {THRESHOLD}")

# 1. Generate the shares
all_shares = generate_shares(SECRET_VALUE, NUM_SHARES, THRESHOLD)
print("\nGenerated Shares (x, y):")
for share in all_shares:
    print(f"  - {share}")

# 2. Reconstruct the secret using 'k' shares
selected_shares = random.sample(all_shares, THRESHOLD)
print(f"\nReconstructing with {THRESHOLD} random shares...")
reconstructed = reconstruct_secret(selected_shares, THRESHOLD)
print(f"Reconstructed Secret: {reconstructed}")
assert reconstructed == SECRET_VALUE
print("Verification Successful: Reconstructed secret matches the original.")

# 3. Try to reconstruct with fewer than 'k' shares (this should fail)
try:
    not_enough_shares = all_shares[:THRESHOLD - 1]
    print(f"\nAttempting to reconstruct with {len(not_enough_shares)} shares...")
    reconstruct_secret(not_enough_shares, THRESHOLD)
except ValueError as e:
    print(f"Failed as expected: {e}")


--- Part 2: Shamir's Secret Sharing ---
Original Secret: 1234, Total Shares (n): 6, Threshold (k): 3

Generated Shares (x, y):
  - (1, 132493185775773060645496853162756139439)
  - (2, 73616909133814893456567194763879232494)
  - (3, 163653536995063961896585632235137491853)
  - (4, 62320702438581802502177558144762706062)
  - (5, 109900772385306878736717579924523086575)
  - (6, 136252563374769958868518393858534527665)

Reconstructing with 3 random shares...
Reconstructed Secret: 1234
Verification Successful: Reconstructed secret matches the original.

Attempting to reconstruct with 2 shares...
Failed as expected: Need at least 3 shares to reconstruct the secret.


In [ ]:

# File Name: MPC_Lab_Sarah_Saleh_Part3.py
# Student Name: Sarah Saleh
# Description: Part III - Garbled Circuits Example (Logic Gate)

# =============================================================================


import random

class GarbledCircuitAND:
    """
    Implements a garbled circuit for AND gate using secure multi-party computation
    This allows two parties to compute AND operation without revealing their inputs
    """

    def __init__(self):
        # Generate random labels (keys) for each possible input/output value
        # Each wire (A, B, C) has two labels: one for value 0 and one for value 1
        self.labels = {
            'A0': random.getrandbits(128),  # Label for A=0
            'A1': random.getrandbits(128),  # Label for A=1
            'B0': random.getrandbits(128),  # Label for B=0
            'B1': random.getrandbits(128),  # Label for B=1
            'C0': random.getrandbits(128),  # Label for output=0
            'C1': random.getrandbits(128)   # Label for output=1
        }

        # Create the garbled table - this is the encrypted truth table
        self.garbled_table = self._create_garbled_table()

    def _create_garbled_table(self):
        """
        Creates the garbled table for AND gate
        The table contains encrypted output labels for all input combinations
        """
        table = []

        # All possible input combinations for AND gate
        inputs_combinations = [
            (0, 0),  # A=0, B=0 → Output=0
            (0, 1),  # A=0, B=1 → Output=0
            (1, 0),  # A=1, B=0 → Output=0
            (1, 1)   # A=1, B=1 → Output=1
        ]

        # For each input combination, create an encrypted entry
        for a_val, b_val in inputs_combinations:
            # Get the input labels for this combination
            label_a = self.labels[f'A{a_val}']
            label_b = self.labels[f'B{b_val}']

            # Calculate the expected output (AND operation)
            output_val = 1 if (a_val and b_val) else 0
            label_output = self.labels[f'C{output_val}']

            # Encrypt the output label using input labels (simple XOR encryption)
            encrypted = label_a ^ label_b ^ label_output
            table.append(encrypted)

        # Shuffle the table to hide the order of entries
        random.shuffle(table)
        return table

    def get_input_label(self, input_name, value):
        """
        Returns the encrypted label for a given input value
        Args:
            input_name: 'A' or 'B' (which party's input)
            value: 0 or 1 (the actual input value)
        Returns:
            Encrypted label for that input value
        """
        return self.labels[f'{input_name}{value}']

    def evaluate(self, label_a, label_b):
        """
        Evaluates the garbled circuit using encrypted input labels
        Args:
            label_a: Omar's encrypted input label
            label_b: Nancy's encrypted input label
        Returns:
            The result of A AND B (0 or 1)
        """
        # Try to decrypt each entry in the garbled table
        for encrypted in self.garbled_table:
            # Attempt decryption using both input labels
            potential_output = label_a ^ label_b ^ encrypted

            # Check if the decrypted value is a valid output label
            if potential_output == self.labels['C0']:
                return 0  # Output is 0
            elif potential_output == self.labels['C1']:
                return 1  # Output is 1

        # If no valid output found, raise error
        raise ValueError("Cannot determine output - invalid input labels")

    def run_example(self, a_input, b_input):
        """
        Runs a complete example with given inputs
        Args:
            a_input: Omar's secret input (0 or 1)
            b_input: Nancy's secret input (0 or 1)
        """
        print(f"Secret Inputs: Omar(A)={a_input}, Nancy(B)={b_input}")
        print(f"Expected Output: A AND B = {a_input and b_input}")

        # Step 1: Each party gets their encrypted label
        label_a = self.get_input_label('A', a_input)
        label_b = self.get_input_label('B', b_input)

        print(f"Omar's encrypted label: {label_a:032x}")
        print(f"Nancy's encrypted label: {label_b:032x}")

        # Step 2: Evaluate the circuit (can be done by a third party)
        result = self.evaluate(label_a, label_b)

        print(f"Computed Result: {result}")
        print(f"Calculation Correct: {result == (a_input and b_input)}")
        print("-" * 50)

        return result

def demonstrate_mpc_security():
    """
    Demonstrates the security properties of the garbled circuit
    Shows that individual labels don't reveal the actual inputs
    """
    print("SECURITY DEMONSTRATION")
    print("=" * 50)

    # Create a circuit
    circuit = GarbledCircuitAND()

    # Show that labels for 0 and 1 look completely random
    print("Random appearance of labels (no information leakage):")
    print(f"Label for A=0: {circuit.labels['A0']:032x}")
    print(f"Label for A=1: {circuit.labels['A1']:032x}")
    print("→ Cannot determine actual input value from label alone")
    print("-" * 50)

# Main execution
if __name__ == "__main__":
    print("GARBLED CIRCUIT - AND GATE IMPLEMENTATION")
    print("Secure Multi-Party Computation Lab")
    print("=" * 60)

    # Test all possible input combinations
    print("\nTESTING ALL INPUT COMBINATIONS:")
    print("=" * 40)

    test_cases = [(0,0), (0,1), (1,0), (1,1)]

    for a, b in test_cases:
        # Create new circuit for each test (fresh labels)
        circuit = GarbledCircuitAND()
        circuit.run_example(a, b)

    # Demonstrate security properties
    demonstrate_mpc_security()

    print("\nHOW IT WORKS:")
    print("1. Circuit creator generates random labels for all wires")
    print("2. Creates encrypted truth table (garbled table)")
    print("3. Each party receives only their input labels")
    print("4. Third party evaluates using encrypted labels")
    print("5. Output is revealed without exposing individual inputs")
    print("6. Security: Labels appear random, no input information leaked")

GARBLED CIRCUIT - AND GATE IMPLEMENTATION
Secure Multi-Party Computation Lab

TESTING ALL INPUT COMBINATIONS:
Secret Inputs: Omar(A)=0, Nancy(B)=0
Expected Output: A AND B = 0
Omar's encrypted label: 3773437f0043c8eb3e2774cf3df897ec
Nancy's encrypted label: 4ddcc4af159bd271531dfe4008b63e3a
Computed Result: 0
Calculation Correct: True
--------------------------------------------------
Secret Inputs: Omar(A)=0, Nancy(B)=1
Expected Output: A AND B = 0
Omar's encrypted label: dcdfa0068e3f7027f8c36e60b0425beb
Nancy's encrypted label: 46b7ab0a7aad08f221a90167ececdef7
Computed Result: 0
Calculation Correct: True
--------------------------------------------------
Secret Inputs: Omar(A)=1, Nancy(B)=0
Expected Output: A AND B = 0
Omar's encrypted label: 000a00b80b88ffdba1a98161a8faa217
Nancy's encrypted label: c72fcc58714f6c126d39f2652641bc91
Computed Result: 0
Calculation Correct: True
--------------------------------------------------
Secret Inputs: Omar(A)=1, Nancy(B)=1
Expected Output: A AND